In [6]:
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
from tabulate import tabulate
import tensorflow._api.v2.compat.v1 as tf


import algorithms.heartrate as hr
import utils

In [7]:
# tensorflow settings
tf.logging.set_verbosity(tf.logging.ERROR)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.keras.backend.set_session(tf.Session(config=config))

# disable warnings
import warnings
warnings.filterwarnings("ignore")

In [21]:
# x_data_train, y_data_train, groups_train = ...
# x_data_test, y_data_test, groups_test = ...

# dummy:
train_size, test_size = 774, 258
n_groups_train, n_groups_test = 27, 9
data = np.genfromtxt('C:/Users/dario/Desktop/pr1/1000x400/rawprova1/raw_2022-03-14_bcg.csv', delimiter=';')
data = data[:len(data)-len(data)%400]
data = data.reshape( int(len(data)/400), 400)
x_data_train = np.expand_dims(data, axis=2)
data = np.genfromtxt('C:/Users/dario/Desktop/pr1/1000x400/gold/gold.csv', delimiter=',')
data = data[:len(data)-len(data)%16]
y_data_train = np.mean(data.reshape(int(len(data)/16), 16), axis=1)
y_data_train = y_data_train[:774]
groups_train = 27

x_data_test = x_data_train[200:458]
y_data_test = y_data_train[200:458]
groups_test = 9

In [22]:
modelnames = [
    "models.deeper_fcn-x1",
    #"models.stacked_cnn_rnn_improved-x1",
]

In [23]:
predictions_dl_train = {"true": y_data_train, "groups": groups_train}
predictions_dl_test = {"true": y_data_test, "groups": groups_test}

In [24]:
def get_modelpath(modelname):
    return os.path.join("output", modelname)  # modify this if necessary

def get_predictions(modelname, weights_format="weights-{:02d}.h5",
                    batch_size=32, train=True):
    modelpath = get_modelpath(modelname)
    model = utils.get_model_from_json(modelpath)
    
    if not train:
        model.load_weights(os.path.join(modelpath, "final",
                                        weights_format.format(0)))
        y_pred = model.predict(x_data_test)
        tf.keras.backend.clear_session()
        return y_pred[:, 0]

    splitter = utils.get_group_splitter(n_groups_train, groups_train)
    results = []
    for i, (_, v_inds) in enumerate(splitter):
        model.load_weights(os.path.join(modelpath, weights_format.format(i)))
        y_pred = model.predict(x_data_train[v_inds], batch_size=batch_size)
        results = np.r_[results, y_pred[:, 0]]  # append new predictions
    tf.keras.backend.clear_session()
    return results

In [25]:
utils.get_model_from_json(get_modelpath(modelnames[-1])).summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 400, 1)]          0         
                                                                 
 conv1d_11 (Conv1D)          (None, 390, 8)            96        
                                                                 
 batch_normalization_8 (Batc  (None, 390, 8)           32        
 hNormalization)                                                 
                                                                 
 leaky_re_lu_10 (LeakyReLU)  (None, 390, 8)            0         
                                                                 
 conv1d_12 (Conv1D)          (None, 190, 8)            712       
                                                                 
 batch_normalization_9 (Batc  (None, 190, 8)           32        
 hNormalization)                                           

In [26]:
for modelname in modelnames:
    predictions_dl_train[modelname] = get_predictions(modelname)
    predictions_dl_test[modelname] = get_predictions(modelname, train=False)
    print(modelname, hr.hr_mae(y_data_test, predictions_dl_test[modelname]))

TypeError: Singleton array array(27) cannot be considered a valid collection.

In [ ]:
# calculate MAE and MAPE patient-wise (for training and testing set)
dl_results_train = np.zeros((len(modelnames), 1, 2, n_groups_train))
dl_results_test = np.zeros((len(modelnames), 1, 2, n_groups_test))

for res, preds_ in [(dl_results_train, predictions_dl_train),
                    (dl_results_test, predictions_dl_test)]:
    y_true, preds = preds_["true"], preds_
    for ni, name in enumerate(modelnames):
        y_pred = preds[name]
        for gi, group in enumerate(np.unique(preds["groups"])):
            yt = y_true[preds["groups"] == group]
            yp = y_pred[preds["groups"] == group]
            res[ni, 0, :, gi] = (hr.hr_mape(yt, yp), hr.hr_mae(yt, yp))

score_index = 1  # 0: MAPE, 1: MAE
rows = []
for ni, name in enumerate(modelnames):
    errmean = dl_results_test.mean(axis=-1)[ni, 0, score_index]
    errstd = dl_results_test.std(axis=-1)[ni, 0, score_index]
    rows.append((modelname, errmean, errstd))

print("Results on testing data:")
print("========================\n")
print(tabulate(rows, tablefmt="presto", floatfmt=".3f",
               headers=["modelname", "mae" if score_index==1 else "mape", "std"]))